In [1]:
import numpy as np
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
lgcg_path = os.path.abspath(os.path.join('../lgcg'))
if lgcg_path not in sys.path:
    sys.path.append(lgcg_path)
from lib.measure import Measure
from lgcg_stochastic import LGCG_stochastic
from lgcg import LGCG

# Generate Data and Define Functions

In [2]:
Omega = np.array([[0,1],[0,1]])
alpha = 0.1
observation_resolution = 5
observations = (np.array(np.meshgrid(
                    *(
                        np.linspace(bound[0], bound[1], observation_resolution)
                        for bound in Omega
                    ))
            ).reshape(len(Omega), -1).T)
std_factor = 4
k = lambda x: np.array([np.exp(-np.linalg.norm(point-x)**2/std_factor)/np.sqrt(std_factor*np.pi)**Omega.shape[0] for point in observations])
true_sources = np.array([[0.1,0.3], [0.25,0.8], [0.7,0.5]])
true_weights = np.array([5,-3,7])
target = sum([weight*k(source) for weight, source in zip(true_weights, true_sources)])
target = target/np.linalg.norm(target)
norm_K_star = np.sqrt(len(observations)/(std_factor*np.pi)**Omega.shape[0])
norm_K_star_L = np.sqrt(0.5*std_factor*len(observations)/(np.pi*std_factor)**Omega.shape[0])

In [3]:
g = lambda u: alpha * np.linalg.norm(u, ord=1)
f = lambda u: 0.5 * np.linalg.norm(sum(c * k(x) for x, c in zip(u.support, u.coefficients)) - target) ** 2

In [4]:
def p_raw(u, k, target):
    def p_u(x):
        Ku = sum(k(x)*c for c, x in zip(u.coefficients, u.support))
        inner = Ku-target
        return -np.dot(inner, k(x))
    return p_u

p = lambda u: p_raw(u, k=k, target=target)

In [5]:
def grad_k_raw(observations, std_factor):
    def grad_k(x):
        return np.array([2*(point-x)*np.exp(-np.linalg.norm(point-x)**2/std_factor)/(std_factor*np.sqrt(std_factor*np.pi)**Omega.shape[0]) for point in observations])
    return grad_k

grad_k = grad_k_raw(observations, std_factor) # The Jacobian of k, shape=(len(observations), Omega.shape[0])

In [6]:
def hess_k_raw(observations, std_factor):
    def hess_k(x):
        first_part = lambda point: -2*np.eye(Omega.shape[0])*np.exp(-np.linalg.norm(point-x)**2/std_factor)/(std_factor*np.sqrt(std_factor*np.pi)**Omega.shape[0])
        second_part = lambda point: 4*np.dot((point-x).reshape(-1,1), np.array([point-x]))*np.exp(-np.linalg.norm(point-x)**2/std_factor)/(std_factor**2*np.sqrt(std_factor*np.pi)**Omega.shape[0])
        return np.array([first_part(point)+second_part(point) for point in observations])
    return hess_k

hess_k = hess_k_raw(observations, std_factor) # The derivative of the Jacobian of k, shape=(len(observations), Omega.shape[0], Omega.shape[0])

In [7]:
def grad_P_raw(k, target, observations, std_factor):
    grad_k = grad_k_raw(observations, std_factor)
    def grad_P(x,u):
        p_u = p_raw(u, k, target)
        inner = target-sum(k(x)*c for c, x in zip(u.coefficients, u.support))
        return np.sign(p_u(x))*np.matmul(grad_k(x).T, inner)
    return grad_P

grad_P = grad_P_raw(k, target, observations, std_factor)

In [8]:
def hess_P_raw(k, target, observations, std_factor):
    hess_k = hess_k_raw(observations, std_factor)
    def hess_P(x,u):
        p_u = p_raw(u, k, target)
        inner = target-sum(k(x)*c for c, x in zip(u.coefficients, u.support))
        return np.sign(p_u(x))*np.tensordot(hess_k(x),inner,axes=([0,0]))
    return hess_P

hess_P = hess_P_raw(k, target, observations, std_factor)

In [9]:
def get_grad_j(k, grad_k, alpha, target):
    def grad_j(positions, coefs):
        to_return = []
        grad_F = (
            np.sum(
                np.array([c * k(x) for x, c in zip(positions, coefs)]),
                axis=0,
            )
            - target
        )
        for ind, x in enumerate(positions):
            # nabla_x_ind
            array = coefs[ind] * np.matmul(grad_k(x).T, grad_F)
            to_return += array.tolist()
        # nabla_u
        K = np.array([k(x) for x in positions])
        to_return += (np.dot(K, grad_F) + alpha * np.sign(coefs)).tolist()
        return np.array(to_return)
    return grad_j

grad_j = get_grad_j(k, grad_k, alpha, target)

In [10]:
def get_hess_j(k, grad_k, hess_k, target):
    def hess_j(positions, coefs):
        matrix_dimension = len(positions)*Omega.shape[0] + len(coefs)
        hesse_matrix = np.zeros((matrix_dimension, matrix_dimension))
        step = Omega.shape[0]
        coefs_delay = step*len(positions)
        inner = sum(k(x)*c for c, x in zip(coefs, positions))-target
        for i, position in enumerate(positions):
            # nabla_{x_i,x_j}
            for j, other_position in enumerate(positions):
                if j<i:
                    continue
                block = coefs[i]*coefs[j]*np.matmul(grad_k(position).T, grad_k(other_position))
                if i==j:
                    block += coefs[i]*np.tensordot(hess_k(position),inner,axes=([0,0]))
                hesse_matrix[i*step:(i+1)*step, j*step:(j+1)*step] = block
                hesse_matrix[j*step:(j+1)*step, i*step:(i+1)*step] = block.T
            # nabla_{x_i,u_j}
            for j, coef in enumerate(coefs):
                block = coefs[i]*np.matmul(grad_k(position).T, k(positions[j]))
                if i == j:
                    block += np.matmul(grad_k(position).T, inner)
                hesse_matrix[i*step:(i+1)*step, coefs_delay+j] = block
                hesse_matrix[coefs_delay+j, i*step:(i+1)*step] = block.T
        for i, coef in enumerate(coefs):
            # nabla_{u_i,u_j}
            for j, other_coef in enumerate(coefs):
                if j<i:
                    continue
                block = np.dot(k(positions[i]), k(positions[j]))
                hesse_matrix[coefs_delay+i,coefs_delay+j] = block
                hesse_matrix[coefs_delay+j,coefs_delay+i] = block
        return hesse_matrix
    return hess_j

hess_j = get_hess_j(k, grad_k, hess_k, target)

# Test LGCG

In [11]:
exp = LGCG(target=target, 
           k=k, 
           g=g, 
           f=f,
           p=p,
           grad_P=grad_P, 
           norm_K_star=norm_K_star,
           norm_K_star_L=norm_K_star_L,
           grad_j=grad_j,
           hess_j=hess_j,
           alpha=alpha,
           Omega=Omega,
           )

In [14]:
exp.solve_newton(tol=1e-3)

DEBUG:root:SSN in 1 dimensions converged in 5 iterations to tolerance 1.263E-03
INFO:root:1: gcg, support: 1, grad_norm:N/A
DEBUG:root:SSN in 2 dimensions converged in 0 iterations to tolerance 6.317E-04
INFO:root:2: gcg, support: 2, grad_norm:0.005700010102418969
DEBUG:root:SSN in 3 dimensions converged in 1 iterations to tolerance 3.158E-04
INFO:root:3: gcg, support: 3, grad_norm:0.006672956528635634
DEBUG:root:SSN in 3 dimensions converged in 0 iterations to tolerance 1.579E-04
INFO:root:4: gcg, support: 3, grad_norm:0.005126414083436322
DEBUG:root:SSN in 3 dimensions converged in 1 iterations to tolerance 7.896E-05
INFO:root:5: gcg, support: 3, grad_norm:0.0051261737202922784
DEBUG:root:SSN in 2 dimensions converged in 0 iterations to tolerance 1.263E-03
INFO:root:6: newton, support: 2, grad_norm:0.003917877271316288
DEBUG:root:SSN in 3 dimensions converged in 1 iterations to tolerance 3.948E-05
INFO:root:7: gcg, support: 3, grad_norm:0.003917872845329079
DEBUG:root:SSN in 3 dimens

KeyboardInterrupt: 

# Test Stochastic

In [3]:
data_path = "../../sisso/src/test/thermal_conductivity_data.csv"
exp = LGCG_stochastic(data_path=data_path, alpha=0.1)

In [4]:
exp.solve(tol=1e-7)

INFO:root:---------------------
INFO:root:---------------------


Time to generate feat space: 262.406 s


INFO:root:0.9755782288434427
INFO:root:1
INFO:root:K []
INFO:root:[0.01650826 0.08398484 0.03875523 0.03877598 0.04318637 0.04194306
 0.04804311 0.06922419 0.08354961 0.11852333 0.04570393 0.04838697
 0.06082152 0.04974988 0.03391667 0.03654056 0.03532187 0.03515929
 0.04248941 0.04185097 0.0362112  0.07187031 0.0379061  0.04252391
 0.03777911 0.13475375 0.12981585 0.11154833 0.34266939 0.21564257
 0.06654163 0.12626913 0.13974483 0.09623659 0.11400463 0.09425901
 0.15303787 0.08798618 0.07927755 0.18412779 0.18997568 0.23295921
 0.08915301 0.20109564 0.13541286 0.09429843 0.07918945 0.06793185
 0.06641488 0.1338539  0.12592129 0.12139781 0.13090409 0.1034475
 0.08834254 0.1011218  0.05419033 0.06345643 0.05942181 0.05300143
 0.0721522  0.09091953 0.08603043 0.10407362 0.1014441  0.065165
 0.19817434 0.27812532 0.24831032 0.06895994 0.13319063 0.11928598
 0.08974987 0.10409334 0.06904784]
/vol/cs-hu/hnatiuar@hu-berlin.de/Dokumente/lazified-generalized-conditional-gradient/lgcg/lgcg_sto

Time to generate feat space: 262.994 s


INFO:root:0.9539958314747843
INFO:root:1
INFO:root:['((omega_Gamma_max / sigma_os) / (Theta_p * L_min_prim))']
INFO:root:2: Phi 3.106E+00, epsilon 6.361E+00, support 1, Psi 5.050E+00
INFO:root:K [[0.01650826 0.08398484 0.03875523 0.03877598 0.04318637 0.04194306
  0.04804311 0.06922419 0.08354961 0.11852333 0.04570393 0.04838697
  0.06082152 0.04974988 0.03391667 0.03654056 0.03532187 0.03515929
  0.04248941 0.04185097 0.0362112  0.07187031 0.0379061  0.04252391
  0.03777911 0.13475375 0.12981585 0.11154833 0.34266939 0.21564257
  0.06654163 0.12626913 0.13974483 0.09623659 0.11400463 0.09425901
  0.15303787 0.08798618 0.07927755 0.18412779 0.18997568 0.23295921
  0.08915301 0.20109564 0.13541286 0.09429843 0.07918945 0.06793185
  0.06641488 0.1338539  0.12592129 0.12139781 0.13090409 0.1034475
  0.08834254 0.1011218  0.05419033 0.06345643 0.05942181 0.05300143
  0.0721522  0.09091953 0.08603043 0.10407362 0.1014441  0.065165
  0.19817434 0.27812532 0.24831032 0.06895994 0.13319063 0.1